# Problem 18
##  Maximum path sum I
------

By starting at the top of the triangle below and moving to adjacent numbers on the row below, the maximum total from top to bottom is 23.

<center>
<div>3</div>
<div>7 4</div>
<div>2 4 6</div>
<div>8 5 9 3</div>
</center>

That is, 3 + 7 + 4 + 9 = 23.

*Find the maximum total from top to bottom of the triangle below:*

<center>
<div>75</div>
<div>95 64</div>
<div>17 47 82</div>
<div>18 35 87 10</div>
<div>20 04 82 47 65</div>
<div>19 01 23 75 03 34</div>
<div>88 02 77 73 07 63 67</div>
<div>99 65 04 28 06 16 70 92</div>
<div>41 41 26 56 83 40 80 70 33</div>
<div>41 48 72 33 47 32 37 16 94 29</div>
<div>53 71 44 65 25 43 91 52 97 51 14</div>
<div>70 11 33 28 77 73 17 78 39 68 17 57</div>
<div>91 71 52 38 17 14 91 43 58 50 27 29 48</div>
<div>63 66 04 68 89 53 67 30 73 16 69 87 40 31</div>
<div>04 62 98 27 23 09 70 98 73 93 38 53 60 04 23</div>
</center>


NOTE: As there are only 16384 routes, it is possible to solve this problem by trying every route. However, Problem 67, is the same challenge with a triangle containing one-hundred rows; it cannot be solved by brute force, and requires a clever method!

---
Correct result: **1074**

### Discussion

As the problem states, while a brute force approach could be used for this particular example, it is not feasible for a pyramid with a number of rows even one order of magnitude larger. This is because the number of possible routes to calculate increases by a factor of 2 for each row $n$ added to the pyramid, meaning the the running time increases as $2^n$.

The best way to reduce the running time is to start at the bottom of the pyramid and calculate the maximum for each sub-tree (or sub-pyramid) moving upward. This was accomplished by creating a Node class which tracks its child nodes, and can get the max (and max-chain) for its own sub-tree. The function which finds the maximum for the pyramid as a whole first creates a representation of the pyramid using these Nodes and then calls *get_sub_tree_max* on the Node at the top of the pyramid. Each of its subnodes does the same until the bottom of the pyramid is reached, at which point the calls begin to return, until execution is back in the first node's method, and the max for the entire pyramid can be calculated by comparing the maxes for its two immediate child nodes.

See also Problem 67 for the use of the same method on a pyramid with 100 rows.

In [1]:
pyramid_str = """
75
95 64
17 47 82
18 35 87 10
20 04 82 47 65
19 01 23 75 03 34
88 02 77 73 07 63 67
99 65 04 28 06 16 70 92
41 41 26 56 83 40 80 70 33
41 48 72 33 47 32 37 16 94 29
53 71 44 65 25 43 91 52 97 51 14
70 11 33 28 77 73 17 78 39 68 17 57
91 71 52 38 17 14 91 43 58 50 27 29 48
63 66 04 68 89 53 67 30 73 16 69 87 40 31
04 62 98 27 23 09 70 98 73 93 38 53 60 04 23
"""

pyramid_list = list(map(lambda x: list(map(int, x.split())), pyramid_str.strip().split('\n')))


class Node(object):

    def __init__(self, value, sub_link_1=None, sub_link_2=None):
        self.value = value
        if sub_link_1 is not None and sub_link_2 is not None:
            self.sub_link_1 = sub_link_1
            self.sub_link_2 = sub_link_2
        else:
            self.sub_link_1 = None
            self.sub_link_2 = None
        self.sub_tree_max = None
        self.sub_tree_chain = [self.value]

    def get_sub_tree_max(self):
        if self.sub_tree_max is None:
            if self.sub_link_1 is not None and self.sub_link_2 is not None:
                sub_max_1 = self.sub_link_1.get_sub_tree_max()
                sub_max_2 = self.sub_link_2.get_sub_tree_max()
                self.sub_tree_max = self.value + max(sub_max_1, sub_max_2)
                self.sub_tree_chain += self.sub_link_1.sub_tree_chain if sub_max_1 > sub_max_2 else self.sub_link_2.sub_tree_chain
            else:
                self.sub_tree_max = self.value
        return self.sub_tree_max

        
def find_max_sum(pyramid):
    # Populating pyramid:
    node_pyramid = []
    for i in range(len(pyramid) - 1, -1, -1):
        newrow = []
        for j in range(0, len(pyramid[i])):
            if i == len(pyramid) - 1:
                node = Node(pyramid[i][j])
            else:
                node = Node(pyramid[i][j], node_pyramid[-1][j], node_pyramid[-1][j + 1])
            newrow.append(node)
        node_pyramid.append(newrow)
    
    # Finding max sum:
    return node_pyramid[::-1][0][0].get_sub_tree_max()


In [2]:
# Running and timing the alternative approaches:
from utils import computation_timer

results = computation_timer({'name':'Sub-tree method', 'func': lambda: find_max_sum(pyramid_list)})

print("Timed Results:")
for result in results:
    print("\t%s:" % result['name'])
    print("\t\tResult: %d, obtained in %f seconds" % (result['result'], result['running_time']))

Timed Results:
	Sub-tree method:
		Result: 1074, obtained in 0.000227 seconds
